# Amazon API Data Extraction Test (OffersV2 Migration)

This notebook tests the **OffersV2** API migration for Amazon product discovery:

## Changes from V1 to V2:
- **Resources**: `OFFERS_*` → `OFFERSV2_*`
- **Response**: `offers` → `offers_v2`
- **Price structure**: `price.amount` → `price.money.amount`
- **New fields**: `is_buy_box_winner`, `deal_details`, `availability`

## Steps:
1. **Check SDK version** - Verify OffersV2 resources are available
2. **`SearchItems`** - Get candidate ASINs with OffersV2
3. **`GetItems`** - Enrich ASINs with detailed OffersV2 data

⚠️ **Deadline**: OffersV1 will stop working after **January 30, 2026**


In [ ]:
# Cell 1: Install Dependencies & Imports

import sys
!{sys.executable} -m pip install python-dotenv python-amazon-paapi>=1.2.3 gspread google-auth-oauthlib pandas openpyxl

import json
import time
from dotenv import load_dotenv

# Load credentials from .env file
load_dotenv()

# Import the specific Amazon PA API components we need
from amazon_paapi.sdk.api.default_api import DefaultApi
from amazon_paapi.sdk.models.partner_type import PartnerType
from amazon_paapi.sdk.models.search_items_request import SearchItemsRequest
from amazon_paapi.sdk.models.search_items_resource import SearchItemsResource
from amazon_paapi.sdk.models.get_items_request import GetItemsRequest
from amazon_paapi.sdk.models.get_items_resource import GetItemsResource
from amazon_paapi.sdk.models.delivery_flag import DeliveryFlag
from amazon_paapi.sdk.rest import ApiException

print("✅ Dependencies and imports are ready.")


✅ Dependencies and imports are ready.


In [14]:
# Cell 2: Configuration and API Client Setup (OffersV2)

from config import conf

# --- PAAPI Credentials & Configuration ---
ACCESS_KEY = conf.amazon.access_key
SECRET_KEY = conf.amazon.secret_key
PARTNER_TAG = conf.amazon.associate_tag
HOST = "webservices.amazon.it"
REGION = "eu-west-1"

# --- API Functions ---

def search_api(browse_node_id, item_page=1, min_price=None, max_price=None, 
               delivery_flags=None, min_reviews_rating=None, min_saving_percent=None):
    """
    Calls the SearchItems API for a given category and page with optional filters.
    Uses OffersV2 resources (new API version).
    
    Args:
        browse_node_id: Browse node ID for the category
        item_page: Page number (default: 1)
        min_price: Minimum price in euros (float, e.g., 10.50)
        max_price: Maximum price in euros (float, e.g., 100.00)
        delivery_flags: List of delivery flags (e.g., ["FulfilledByAmazon"])
        min_reviews_rating: Minimum star rating (int, 1-5, e.g., 4)
        min_saving_percent: Minimum discount percentage (int, e.g., 10 for 10%)
    """
    api_client = DefaultApi(access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST, region=REGION)
    
    # OffersV2 resources for SearchItems
    resources = [
        SearchItemsResource.ITEMINFO_TITLE,
        SearchItemsResource.IMAGES_PRIMARY_LARGE,
        SearchItemsResource.IMAGES_VARIANTS_LARGE,
        SearchItemsResource.CUSTOMERREVIEWS_COUNT,
        SearchItemsResource.CUSTOMERREVIEWS_STARRATING,
        SearchItemsResource.BROWSENODEINFO_WEBSITESALESRANK,
        SearchItemsResource.BROWSENODEINFO_BROWSENODES,
    ]
    
    # Try to add OffersV2 resources if available in SDK
    offersv2_resources = [
        'OFFERSV2_SUMMARIES_LOWESTPRICE',
        'OFFERSV2_LISTINGS_PRICE',
        'OFFERSV2_LISTINGS_AVAILABILITY',
        'OFFERSV2_LISTINGS_CONDITION',
        'OFFERSV2_LISTINGS_ISBUYBOXWINNER',
    ]
    for res_name in offersv2_resources:
        if hasattr(SearchItemsResource, res_name):
            resources.append(getattr(SearchItemsResource, res_name))
            print(f"✅ Added SearchItemsResource.{res_name}")
        else:
            # Fallback to V1 if V2 not available
            v1_name = res_name.replace('OFFERSV2', 'OFFERS')
            if hasattr(SearchItemsResource, v1_name):
                resources.append(getattr(SearchItemsResource, v1_name))
                print(f"⚠️ OffersV2 not available, using fallback: {v1_name}")
    
    # Convert price to cents for API
    min_price_cents = int(min_price * 100) if min_price else None
    max_price_cents = int(max_price * 100) if max_price else None
    
    # Handle delivery flags
    delivery_flag_list = []
    if delivery_flags:
        if "FulfilledByAmazon" in delivery_flags:
            delivery_flag_list.append(DeliveryFlag.FULFILLEDBYAMAZON)
    
    try:
        request = SearchItemsRequest(
            partner_tag=PARTNER_TAG,
            partner_type=PartnerType.ASSOCIATES,
            marketplace="www.amazon.it",
            browse_node_id=browse_node_id,
            item_page=item_page,
            item_count=10,
            sort_by="Featured",
            resources=resources,
            min_price=min_price_cents,
            max_price=max_price_cents,
            min_reviews_rating=min_reviews_rating,
            min_saving_percent=min_saving_percent,
            delivery_flags=delivery_flag_list if delivery_flag_list else None
        )
        response = api_client.search_items(request)
        return response.to_dict()
    except ApiException as e:
        print(f"API Error on page {item_page} for node {browse_node_id}: {e.reason}")
        return None

def get_items_details(asins):
    """Fetches detailed information for a list of ASINs using the GetItems API with OffersV2."""
    if not asins:
        return []
    
    print(f"\n--- Enriching {len(asins)} items with GetItems API (OffersV2) ---")
    api_client = DefaultApi(access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST, region=REGION)
    
    # Build resources list safely - only add resources that exist
    resources = []
    
    # Core resources (non-Offers)
    core_resources = [
        'ITEMINFO_TITLE',
        'IMAGES_PRIMARY_LARGE',
        'IMAGES_VARIANTS_LARGE',
        'CUSTOMERREVIEWS_COUNT',
        'CUSTOMERREVIEWS_STARRATING',
        'ITEMINFO_FEATURES',
        'BROWSENODEINFO_WEBSITESALESRANK'
    ]
    
    # OffersV2 resources (new API) with V1 fallbacks
    offersv2_resources = [
        ('OFFERSV2_LISTINGS_PRICE', 'OFFERS_LISTINGS_PRICE'),
        ('OFFERSV2_LISTINGS_AVAILABILITY', 'OFFERS_LISTINGS_AVAILABILITY'),
        ('OFFERSV2_LISTINGS_CONDITION', 'OFFERS_LISTINGS_CONDITION'),
        ('OFFERSV2_LISTINGS_ISBUYBOXWINNER', None),  # No V1 equivalent
        ('OFFERSV2_LISTINGS_SAVINGBASIS', 'OFFERS_LISTINGS_SAVINGBASIS'),
    ]
    
    # Optional resources - try to add them if they exist
    optional_resources = [
        'ITEMINFO_PRODUCT_INFO',
        'ITEMINFO_TECHNICAL_INFO',
        'ITEMINFO_CONTENT_INFO',
        'ITEMINFO_CLASSIFICATIONS',
        'ITEMINFO_BYLINE_INFO',
        'ITEMINFO_EXTERNAL_IDS',
        'ITEMINFO_MANUFACTURE_INFO',
        'ITEMINFO_TRADE_IN_INFO'
    ]
    
    # Add core resources
    for resource_name in core_resources:
        try:
            resource = getattr(GetItemsResource, resource_name)
            resources.append(resource)
        except AttributeError:
            print(f"⚠️ Warning: Resource {resource_name} not available")
    
    # Add OffersV2 resources (with V1 fallback)
    for v2_name, v1_fallback in offersv2_resources:
        if hasattr(GetItemsResource, v2_name):
            resources.append(getattr(GetItemsResource, v2_name))
            print(f"✅ Using OffersV2: {v2_name}")
        elif v1_fallback and hasattr(GetItemsResource, v1_fallback):
            resources.append(getattr(GetItemsResource, v1_fallback))
            print(f"⚠️ OffersV2 not available, using V1: {v1_fallback}")
    
    # Try to add optional resources
    for resource_name in optional_resources:
        try:
            resource = getattr(GetItemsResource, resource_name)
            resources.append(resource)
        except AttributeError:
            pass  # Silently skip if not available

    asin_chunks = [asins[i:i + 10] for i in range(0, len(asins), 10)]
    enriched_items = []

    for i, chunk in enumerate(asin_chunks):
        print(f"Fetching details for batch {i+1}/{len(asin_chunks)}...")
        request = GetItemsRequest(
            partner_tag=PARTNER_TAG,
            partner_type=PartnerType.ASSOCIATES,
            marketplace="www.amazon.it",
            item_ids=chunk,
            resources=resources
        )
        try:
            response = api_client.get_items(request).to_dict()
            if response and response.get("items_result") and response["items_result"].get("items"):
                enriched_items.extend(response["items_result"]["items"])
        except ApiException as e:
            print(f"API Error (GetItems) on batch {i+1}: {e.reason}")
        
        time.sleep(1)

    return enriched_items

def extract_price_from_item(item):
    """
    Extract price and savings from item, supporting both OffersV1 and OffersV2.
    Returns (price, savings) tuple.
    """
    price = "N/A"
    savings = "N/A"
    
    # Try OffersV2 first (new API)
    if item.get("offers_v2"):
        offers_v2 = item["offers_v2"]
        # OffersV2 Listings
        if offers_v2.get("listings") and len(offers_v2["listings"]) > 0:
            listing = offers_v2["listings"][0]
            if listing.get("price"):
                price_obj = listing["price"]
                # OffersV2 uses nested "money" structure
                if price_obj.get("money"):
                    price = price_obj["money"].get("display_amount", "N/A")
                else:
                    price = price_obj.get("display_amount", "N/A")
                # Savings
                if price_obj.get("savings"):
                    sav = price_obj["savings"]
                    if sav.get("money"):
                        savings = f"{sav['money'].get('display_amount', 'N/A')} ({sav.get('percentage', '')}%)"
                    else:
                        savings = sav.get("display_amount", "N/A")
        # OffersV2 Summaries
        elif offers_v2.get("summaries") and len(offers_v2["summaries"]) > 0:
            summary = offers_v2["summaries"][0]
            if summary.get("lowest_price"):
                lp = summary["lowest_price"]
                if lp.get("money"):
                    price = lp["money"].get("display_amount", "N/A")
                else:
                    price = lp.get("display_amount", "N/A")
    
    # Fallback to OffersV1 (old API)
    elif item.get("offers"):
        offers = item["offers"]
        # V1 Listings
        if offers.get("listings") and len(offers["listings"]) > 0:
            listing = offers["listings"][0]
            if listing.get("price"):
                price = listing["price"].get("display_amount", "N/A")
                if listing["price"].get("savings"):
                    savings = listing["price"]["savings"].get("display_amount", "N/A")
        # V1 Summaries
        elif offers.get("summaries") and len(offers["summaries"]) > 0:
            summary = offers["summaries"][0]
            if summary.get("lowest_price"):
                price = summary["lowest_price"].get("display_amount", "N/A")
            if summary.get("saving"):
                savings = summary["saving"].get("display_amount", "N/A")
    
    return price, savings


def print_search_results(search_response):
    """Pretty print the search results from SearchItems API (supports OffersV1 & V2)."""
    if not search_response:
        print("❌ No response received from API.")
        return
    
    if "search_result" not in search_response:
        print("❌ No search_result in response.")
        print(f"Response keys: {list(search_response.keys())}")
        return
    
    search_result = search_response["search_result"]
    
    total_results = search_result.get("total_result_count", "Unknown")
    search_index = search_result.get("search_index", "Unknown")
    browse_node_id = search_result.get("browse_node", {}).get("id", "Unknown") if search_result.get("browse_node") else "Unknown"
    browse_node_name = search_result.get("browse_node", {}).get("display_name", "Unknown") if search_result.get("browse_node") else "Unknown"
    
    print(f"\n{'='*80}")
    print(f"📊 Search Results Summary (OffersV2 Compatible)")
    print(f"{'='*80}")
    print(f"Total Results: {total_results}")
    print(f"Search Index: {search_index}")
    print(f"Browse Node ID: {browse_node_id}")
    print(f"Browse Node Name: {browse_node_name}")
    print(f"{'='*80}\n")
    
    items = search_result.get("items", [])
    if not items:
        print("⚠️  No items found in search results.")
        return
    
    print(f"Found {len(items)} items on this page:\n")
    
    for idx, item in enumerate(items, 1):
        asin = item.get("asin", "N/A")
        title = item.get("item_info", {}).get("title", {}).get("display_value", "N/A")
        
        # Use universal price extraction
        price, savings = extract_price_from_item(item)
        
        image_url = "N/A"
        if item.get("images") and item["images"].get("primary") and item["images"]["primary"].get("large"):
            image_url = item["images"]["primary"]["large"].get("url", "N/A")
        
        review_count = "N/A"
        star_rating = "N/A"
        if item.get("customer_reviews"):
            review_count = item["customer_reviews"].get("count", "N/A")
            if item["customer_reviews"].get("star_rating"):
                star_rating = item["customer_reviews"]["star_rating"].get("display_value", "N/A")
        
        sales_rank = "N/A"
        if item.get("browse_node_info") and item["browse_node_info"].get("website_sales_rank"):
            sales_rank_obj = item["browse_node_info"]["website_sales_rank"]
            if isinstance(sales_rank_obj, dict):
                sales_rank = sales_rank_obj.get("sales_rank", "N/A")
        
        browse_nodes = "N/A"
        if item.get("browse_node_info") and item["browse_node_info"].get("browse_nodes"):
            browse_nodes_list = item["browse_node_info"]["browse_nodes"]
            if browse_nodes_list:
                browse_nodes = browse_nodes_list[0].get("display_name", "N/A")
        
        detail_url = item.get("detail_page_url", "N/A")
        
        print(f"{'─'*80}")
        print(f"Item #{idx}")
        print(f"{'─'*80}")
        print(f"ASIN:         {asin}")
        print(f"Title:        {title}")
        print(f"Price:        {price}")
        if savings != "N/A":
            print(f"Savings:      {savings}")
        print(f"Reviews:      {review_count} ({star_rating} stars)")
        print(f"Sales Rank:   {sales_rank}")
        print(f"Browse Node:  {browse_nodes}")
        if len(image_url) > 80:
            print(f"Image URL:    {image_url[:77]}...")
        else:
            print(f"Image URL:    {image_url}")
        if len(detail_url) > 80:
            print(f"Detail URL:   {detail_url[:77]}...")
        else:
            print(f"Detail URL:   {detail_url}")
        print()
    
    print(f"{'='*80}\n")

def print_enriched_items(enriched_items):
    """Pretty print the enriched items from GetItems API (supports OffersV1 & V2)."""
    if not enriched_items:
        print("❌ No enriched items to display.")
        return
    
    print(f"\n{'='*80}")
    print(f"📦 Enriched Items Details (OffersV2 Compatible)")
    print(f"{'='*80}")
    print(f"Total Items: {len(enriched_items)}\n")
    
    for idx, item in enumerate(enriched_items, 1):
        asin = item.get("asin", "N/A")
        title = item.get("item_info", {}).get("title", {}).get("display_value", "N/A")
        
        # Use universal price extraction (supports both V1 and V2)
        price, savings = extract_price_from_item(item)
        
        # Check for OffersV2-specific fields
        is_buy_box_winner = None
        if item.get("offers_v2") and item["offers_v2"].get("listings"):
            listing = item["offers_v2"]["listings"][0]
            is_buy_box_winner = listing.get("is_buy_box_winner")
        
        image_url = "N/A"
        if item.get("images") and item["images"].get("primary") and item["images"]["primary"].get("large"):
            image_url = item["images"]["primary"]["large"].get("url", "N/A")
        
        review_count = "N/A"
        star_rating = "N/A"
        if item.get("customer_reviews"):
            review_count = item["customer_reviews"].get("count", "N/A")
            if item["customer_reviews"].get("star_rating"):
                star_rating = item["customer_reviews"]["star_rating"].get("display_value", "N/A")
        
        sales_rank = "N/A"
        if item.get("browse_node_info") and item["browse_node_info"].get("website_sales_rank"):
            sales_rank_obj = item["browse_node_info"]["website_sales_rank"]
            if isinstance(sales_rank_obj, dict):
                sales_rank = sales_rank_obj.get("sales_rank", "N/A")
        
        detail_url = item.get("detail_page_url", "N/A")
        
        # Features (bullet points) - safely get
        features = []
        try:
            features = item.get("item_info", {}).get("features", {}).get("display_values", [])
        except (AttributeError, KeyError, TypeError):
            pass
        
        # Product Info - safely get
        product_info = {}
        try:
            product_info = item.get("item_info", {}).get("product_info", {}) or {}
        except (AttributeError, KeyError, TypeError):
            pass
        
        color = product_info.get("color", {}).get("display_value", None) if product_info and product_info.get("color") else None
        size = product_info.get("size", {}).get("display_value", None) if product_info and product_info.get("size") else None
        unit_count = product_info.get("unit_count", {}).get("display_value", None) if product_info and product_info.get("unit_count") else None
        
        # Classifications - safely get
        classifications = {}
        try:
            classifications = item.get("item_info", {}).get("classifications", {}) or {}
        except (AttributeError, KeyError, TypeError):
            pass
        
        product_group = classifications.get("product_group", {}).get("display_value", None) if classifications and classifications.get("product_group") else None
        binding = classifications.get("binding", {}).get("display_value", None) if classifications and classifications.get("binding") else None
        
        # Content Info - safely get
        content_info = {}
        try:
            content_info = item.get("item_info", {}).get("content_info", {}) or {}
        except (AttributeError, KeyError, TypeError):
            pass
        
        pages_count = content_info.get("pages_count", {}).get("display_value", None) if content_info and content_info.get("pages_count") else None
        publication_date = content_info.get("publication_date", {}).get("display_value", None) if content_info and content_info.get("publication_date") else None
        
        # Technical Info - safely get
        technical_info = {}
        try:
            technical_info = item.get("item_info", {}).get("technical_info", {}) or {}
        except (AttributeError, KeyError, TypeError):
            pass
        
        technical_info_str = "Available" if technical_info else None
        
        print(f"{'─'*80}")
        print(f"Item #{idx}")
        print(f"{'─'*80}")
        print(f"ASIN:         {asin}")
        print(f"Title:        {title}")
        print(f"Price:        {price}")
        if savings != "N/A":
            print(f"Savings:      {savings}")
        if is_buy_box_winner is not None:
            print(f"Buy Box:      {'✅ Winner' if is_buy_box_winner else '❌ Not winner'}")
        print(f"Reviews:      {review_count} ({star_rating} stars)")
        print(f"Sales Rank:   {sales_rank}")
        if product_group:
            print(f"Product Group: {product_group}")
        if binding:
            print(f"Binding:      {binding}")
        if color:
            print(f"Color:        {color}")
        if size:
            print(f"Size:         {size}")
        if unit_count:
            print(f"Unit Count:   {unit_count}")
        if pages_count:
            print(f"Pages:        {pages_count}")
        if publication_date:
            print(f"Pub Date:     {publication_date}")
        if technical_info_str:
            print(f"Technical Info: {technical_info_str}")
        if features:
            print(f"Features:     {len(features)} bullet points")
            for i, feature in enumerate(features[:3], 1):
                feature_text = feature[:70] + "..." if len(feature) > 70 else feature
                print(f"  {i}. {feature_text}")
            if len(features) > 3:
                print(f"  ... and {len(features) - 3} more")
        print(f"Image URL:    {image_url[:80]}..." if len(image_url) > 80 else f"Image URL:    {image_url}")
        print(f"Detail URL:   {detail_url[:80]}..." if len(detail_url) > 80 else f"Detail URL:   {detail_url}")
        print()
    
    print(f"{'='*80}\n")

print("✅ Configuration and helper functions are defined.")



✅ Configuration and helper functions are defined.


# Функция для получения ВСЕХ данных по ASIN


In [11]:
# Cell: Функция для получения ВСЕХ данных по ASIN
# Показывает полный ответ API для анализа структуры данных

def get_full_item_data(asin: str, print_raw: bool = True):
    """
    Получает ВСЕ доступные данные по ASIN и выводит их.
    
    Args:
        asin: ASIN товара (например, "B0BP2QZLH7")
        print_raw: Если True, выводит полный JSON ответ
    
    Returns:
        dict: Полный ответ API
    """
    api_client = DefaultApi(access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST, region=REGION)
    
    # Собираем ВСЕ доступные ресурсы
    all_resources = []
    
    # Получаем все атрибуты GetItemsResource
    for attr_name in dir(GetItemsResource):
        if not attr_name.startswith('_') and attr_name.isupper():
            try:
                resource = getattr(GetItemsResource, attr_name)
                all_resources.append(resource)
            except:
                pass
    
    print(f"📋 Запрашиваем {len(all_resources)} ресурсов для ASIN: {asin}")
    print(f"{'='*70}\n")
    
    try:
        request = GetItemsRequest(
            partner_tag=PARTNER_TAG,
            partner_type=PartnerType.ASSOCIATES,
            marketplace="www.amazon.it",
            item_ids=[asin],
            resources=all_resources
        )
        
        response = api_client.get_items(request)
        result = response.to_dict()
        
        if result and result.get("items_result") and result["items_result"].get("items"):
            item = result["items_result"]["items"][0]
            
            print(f"✅ Данные получены успешно!")
            print(f"\n{'='*70}")
            print(f"📦 СТРУКТУРА ОТВЕТА ДЛЯ ASIN: {asin}")
            print(f"{'='*70}\n")
            
            # Показываем ключи верхнего уровня
            print("🔑 Ключи верхнего уровня:")
            for key in sorted(item.keys()):
                value = item[key]
                if value is not None:
                    if isinstance(value, dict):
                        print(f"   📁 {key}: {{...}} ({len(value)} ключей)")
                    elif isinstance(value, list):
                        print(f"   📋 {key}: [...] ({len(value)} элементов)")
                    else:
                        print(f"   📄 {key}: {str(value)[:50]}...")
            
            # Детальный вывод каждой секции
            print(f"\n{'─'*70}")
            print("📊 ДЕТАЛЬНЫЕ ДАННЫЕ ПО СЕКЦИЯМ:")
            print(f"{'─'*70}\n")
            
            for key in sorted(item.keys()):
                value = item[key]
                if value is not None:
                    print(f"\n🔹 {key.upper()}:")
                    print(f"   {json.dumps(value, indent=4, ensure_ascii=False)[:2000]}")
                    if len(json.dumps(value)) > 2000:
                        print("   ... (обрезано)")
            
            # Полный JSON если нужен
            if print_raw:
                print(f"\n{'='*70}")
                print("📄 ПОЛНЫЙ RAW JSON:")
                print(f"{'='*70}\n")
                print(json.dumps(item, indent=2, ensure_ascii=False))
            
            return item
        else:
            print(f"❌ Товар не найден: {asin}")
            if result.get("errors"):
                for error in result["errors"]:
                    print(f"   Error: {error}")
            return None
            
    except ApiException as e:
        print(f"❌ API Error: {e.reason}")
        return None
    except Exception as e:
        print(f"❌ Error: {type(e).__name__}: {e}")
        return None


def get_item_summary(asin: str):
    """
    Краткая сводка по товару - только ключевые данные.
    """
    item = get_full_item_data(asin, print_raw=False)
    
    if not item:
        return None
    
    print(f"\n{'='*70}")
    print(f"📊 КРАТКАЯ СВОДКА")
    print(f"{'='*70}\n")
    
    # ASIN
    print(f"ASIN: {item.get('asin', 'N/A')}")
    
    # Title
    title = item.get("item_info", {}).get("title", {}).get("display_value", "N/A")
    print(f"Название: {title}")
    
    # Price (V1 и V2)
    price, savings = extract_price_from_item(item)
    print(f"Цена: {price}")
    if savings != "N/A":
        print(f"Скидка: {savings}")
    
    # Reviews
    if item.get("customer_reviews"):
        cr = item["customer_reviews"]
        count = cr.get("count", "N/A")
        rating = cr.get("star_rating", {}).get("value", "N/A")
        print(f"Отзывы: {count} ({rating}⭐)")
    
    # Sales Rank
    if item.get("browse_node_info", {}).get("website_sales_rank"):
        rank = item["browse_node_info"]["website_sales_rank"].get("sales_rank", "N/A")
        print(f"Sales Rank: #{rank}")
    
    # Images
    if item.get("images"):
        primary = "✅" if item["images"].get("primary") else "❌"
        variants = len(item["images"].get("variants", []) or [])
        print(f"Изображения: Primary {primary}, Variants: {variants}")
    
    # Features
    features = item.get("item_info", {}).get("features", {}).get("display_values", [])
    if features:
        print(f"Характеристики: {len(features)} пунктов")
    
    # URL
    url = item.get("detail_page_url", "N/A")
    print(f"URL: {url[:60]}..." if len(url) > 60 else f"URL: {url}")
    
    return item


# Пример использования:
print("✅ Функции определены!")
print("\n📌 Использование:")
print("   get_full_item_data('B0BP2QZLH7')      # Полный вывод всех данных")
print("   get_full_item_data('B0BP2QZLH7', print_raw=False)  # Без raw JSON")
print("   get_item_summary('B0BP2QZLH7')        # Краткая сводка")


✅ Функции определены!

📌 Использование:
   get_full_item_data('B0BP2QZLH7')      # Полный вывод всех данных
   get_full_item_data('B0BP2QZLH7', print_raw=False)  # Без raw JSON
   get_item_summary('B0BP2QZLH7')        # Краткая сводка


# Тест: Получить все данные по конкретному ASIN


In [15]:
# Вариант 1: Полный вывод со всеми данными
item_data = get_full_item_data("B0FKM2741R")

# Вариант 2: Краткая сводка (без raw JSON)
# item_summary = get_item_summary("B0FMR9K8VJ")


📋 Запрашиваем 59 ресурсов для ASIN: B0FKM2741R

✅ Данные получены успешно!

📦 СТРУКТУРА ОТВЕТА ДЛЯ ASIN: B0FKM2741R

🔑 Ключи верхнего уровня:
   📄 asin: B0FKM2741R...
   📁 browse_node_info: {...} (2 ключей)
   📄 detail_page_url: https://www.amazon.it/dp/B0FKM2741R?tag=cucinacona...
   📁 images: {...} (2 ключей)
   📁 item_info: {...} (11 ключей)
   📁 offers: {...} (2 ключей)
   📄 parent_asin: B0G2RBNKZ5...

──────────────────────────────────────────────────────────────────────
📊 ДЕТАЛЬНЫЕ ДАННЫЕ ПО СЕКЦИЯМ:
──────────────────────────────────────────────────────────────────────


🔹 ASIN:
   "B0FKM2741R"

🔹 BROWSE_NODE_INFO:
   {
    "browse_nodes": [
        {
            "ancestor": {
                "ancestor": {
                    "ancestor": {
                        "ancestor": {
                            "ancestor": null,
                            "context_free_name": "Giardino e giardinaggio",
                            "display_name": "Giardino e giardinaggio",
            

# Сводка

# Экспорт дочерних категорий в Excel


In [21]:
# Функция для экспорта дочерних категорий в Excel

import pandas as pd
from typing import List
from amazon_paapi.sdk.models.get_browse_nodes_request import GetBrowseNodesRequest
from amazon_paapi.sdk.models.get_browse_nodes_resource import GetBrowseNodesResource

def export_children_to_excel(node_ids: List[str], filename: str = "browse_nodes_children.xlsx") -> str:
    """
    Получает дочерние категории для списка Node ID и экспортирует в Excel.
    
    Args:
        node_ids: Список browse_node_id для обработки
        filename: Имя файла Excel (по умолчанию: "browse_nodes_children.xlsx")
    
    Returns:
        Путь к созданному файлу
    """
    api_client = DefaultApi(access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST, region=REGION)
    
    # Собираем все доступные ресурсы для GetBrowseNodes
    resources = []
    for attr in dir(GetBrowseNodesResource):
        if not attr.startswith('_') and attr.isupper():
            resources.append(getattr(GetBrowseNodesResource, attr))
    
    # Список для данных Excel
    excel_data = []
    
    print(f"📋 Обрабатываем {len(node_ids)} категорий...\n")
    
    for idx, node_id in enumerate(node_ids, 1):
        print(f"[{idx}/{len(node_ids)}] Обрабатываем Node ID: {node_id}")
        
        try:
            request = GetBrowseNodesRequest(
                partner_tag=PARTNER_TAG,
                partner_type=PartnerType.ASSOCIATES,
                marketplace="www.amazon.it",
                browse_node_ids=[node_id],
                resources=resources,
                languages_of_preference=["it_IT"]
            )
            
            response = api_client.get_browse_nodes(request)
            result = response.to_dict()
            
            if not result or not result.get("browse_nodes_result"):
                print(f"   ⚠️ Категория не найдена, пропускаем\n")
                continue
            
            nodes_result = result["browse_nodes_result"].get("browse_nodes", [])
            if not nodes_result:
                print(f"   ⚠️ Нет данных, пропускаем\n")
                continue
            
            node = nodes_result[0]
            ancestor_name = node.get("display_name") or node.get("context_free_name", "Unknown")
            
            # Получаем дочерние категории
            children = node.get("children", []) or []
            
            if not children:
                # Если нет детей - добавляем строку с пустым Child Name
                excel_data.append({
                    "Ancestor Name": ancestor_name,
                    "Child Name": "",
                    "Child Node ID": ""
                })
                print(f"   ✅ {ancestor_name} - нет дочерних категорий\n")
            else:
                # Добавляем все дочерние категории
                for child in children:
                    child_id = child.get("id")
                    child_name = child.get("display_name") or child.get("context_free_name", "Unknown")
                    
                    excel_data.append({
                        "Ancestor Name": ancestor_name,
                        "Child Name": child_name,
                        "Child Node ID": child_id
                    })
                
                print(f"   ✅ {ancestor_name} - найдено {len(children)} дочерних категорий\n")
            
            # Небольшая задержка между запросами
            time.sleep(0.5)
            
        except Exception as e:
            print(f"   ❌ Ошибка: {type(e).__name__}: {e}\n")
            continue
    
    # Создаем DataFrame и экспортируем в Excel
    if not excel_data:
        print("❌ Нет данных для экспорта!")
        return ""
    
    df = pd.DataFrame(excel_data)
    df.to_excel(filename, index=False, engine='openpyxl')
    
    print(f"{'='*70}")
    print(f"✅ Excel файл создан: {filename}")
    print(f"📊 Всего строк: {len(excel_data)}")
    print(f"{'='*70}\n")
    
    return filename

print("✅ Функция export_children_to_excel() готова!")


✅ Функция export_children_to_excel() готова!


## Пример использования


In [27]:
# Пример: экспорт дочерних категорий для нескольких Node ID

# Список Node ID для обработки
node_ids_list = [
    "1347370031",  # Abbigliamento
    "15664266031",  
    "2892907031",
    "14704080031",
    "2892908031",
"2892912031",
"2892913031",
"2892915031",
"3950200031",
"2892917031",
"2892918031",
"2892919031",
"2892920031",
"2892922031",
"2892923031",
"2892924031",
"2892904031",
"6306896031",
"27088076031",
"4327880031",
"4327902031",
"6306897031",
"6306898031",
"6306899031",
"6306900031",
"508770031",
"508879031",
"508821031",
"460081031",
"460161031",
"460158031",
"460187031",
"17532506031",
"473287031",
"1465649031",
"473584031",
"460188031",
"49981441031",
"1497227031",
"89757735031",
"5515768031",
"5515769031",
"5518831031",
"5518832031",
"12710833031",
"2892860031",
"12422031031",
"26979646031",
"5512288031",
"5512289031",
"731466031",
"9337408031",
"4380634031",
"4327117031",
"4327135031",
"4327083031",
"4327084031",
"4327085031",
"6691169031",
"4327089031",
"4327088031",
"6394759031",
"2808571031",
"3225815031",
"731676031",
"679995031",
"602473031",
"732997031",
"13713023031",
"4340026031",
"1849538031",
"731678031",
"602474031",
"731677031",
"652535031",
"652615031",
"6311641031",
"10417753031",
"14897362031",
"6571983031",
"6311637031",
"16243687031",
"6311639031",
"6571998031",
"489712031",
"4290081031",
"4290082031",
"4290083031",
"4290087031",
"26039477031",
"4582622031",
"937268031",
"967387031",
"968488031",
"4551672031",
"5160113031",
"26039478031",
"3635984031",
"3119607031",
"731505031",
"3119611031",
"3119612031",
"3119617031",

]

# Экспорт в Excel
export_children_to_excel(node_ids_list, filename="my_categories.xlsx")


📋 Обрабатываем 103 категорий...

[1/103] Обрабатываем Node ID: 1347370031
   ✅ Abbigliamento sportivo - найдено 14 дочерних категорий

[2/103] Обрабатываем Node ID: 15664266031
   ✅ Body - нет дочерних категорий

[3/103] Обрабатываем Node ID: 2892907031
   ✅ Calze e collant - найдено 6 дочерних категорий

[4/103] Обрабатываем Node ID: 14704080031
   ✅ Felpe - найдено 2 дочерних категорий

[5/103] Обрабатываем Node ID: 2892908031
   ✅ Giacche e cappotti - найдено 4 дочерних категорий

[6/103] Обрабатываем Node ID: 2892912031
   ✅ Jeans - нет дочерних категорий

[7/103] Обрабатываем Node ID: 2892913031
   ✅ Leggings - нет дочерних категорий

[8/103] Обрабатываем Node ID: 2892915031
   ✅ Maglioni e cardigan - найдено 6 дочерних категорий

[9/103] Обрабатываем Node ID: 3950200031
   ✅ Mare e piscina - найдено 9 дочерних категорий

[10/103] Обрабатываем Node ID: 2892917031
   ✅ Monopezzi e tutine - нет дочерних категорий

[11/103] Обрабатываем Node ID: 2892918031
   ✅ Neve e pioggia - найде

'my_categories.xlsx'

In [23]:
# Функция для получения расширенного описания товара

def get_product_description_data(asin: str) -> dict:
    """
    Получает все текстовые данные для описания товара по ASIN.
    Features + Title + ProductInfo + TechnicalInfo + ByLineInfo
    """
    api_client = DefaultApi(access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST, region=REGION)
    
    resources = [
        GetItemsResource.ITEMINFO_TITLE,
        GetItemsResource.ITEMINFO_FEATURES,
        GetItemsResource.ITEMINFO_BYLINEINFO,
        GetItemsResource.ITEMINFO_PRODUCTINFO,
        GetItemsResource.ITEMINFO_TECHNICALINFO,
        GetItemsResource.ITEMINFO_CONTENTINFO,
        GetItemsResource.ITEMINFO_CLASSIFICATIONS,
    ]
    
    try:
        request = GetItemsRequest(
            partner_tag=PARTNER_TAG,
            partner_type=PartnerType.ASSOCIATES,
            marketplace="www.amazon.it",
            item_ids=[asin],
            resources=resources
        )
        
        response = api_client.get_items(request)
        result = response.to_dict()
        
        if not result or not result.get("items_result") or not result["items_result"].get("items"):
            print(f"Товар не найден: {asin}")
            return {}
        
        item = result["items_result"]["items"][0]
        item_info = item.get("item_info", {})
        
        data = {"asin": asin, "title": None, "brand": None, "manufacturer": None,
                "features": [], "product_info": {}, "technical_info": {},
                "edition": None, "product_group": None, "binding": None}
        
        # Title
        if item_info.get("title"):
            data["title"] = item_info["title"].get("display_value")
        
        # Brand & Manufacturer
        if item_info.get("by_line_info"):
            bli = item_info["by_line_info"]
            if bli.get("brand"):
                data["brand"] = bli["brand"].get("display_value")
            if bli.get("manufacturer"):
                data["manufacturer"] = bli["manufacturer"].get("display_value")
        
        # Features (буллеты)
        if item_info.get("features"):
            data["features"] = item_info["features"].get("display_values", [])
        
        # ProductInfo (цвет, размер, дата)
        if item_info.get("product_info"):
            pi = item_info["product_info"]
            for key in ["color", "size", "unit_count", "release_date"]:
                if pi.get(key):
                    data["product_info"][key] = pi[key].get("display_value")
        
        # TechnicalInfo (формат)
        if item_info.get("technical_info"):
            ti = item_info["technical_info"]
            if ti.get("formats"):
                data["technical_info"]["formats"] = ti["formats"].get("display_values", [])
        
        # Edition
        if item_info.get("content_info") and item_info["content_info"].get("edition"):
            data["edition"] = item_info["content_info"]["edition"].get("display_value")
        
        # Classifications
        if item_info.get("classifications"):
            cls = item_info["classifications"]
            if cls.get("product_group"):
                data["product_group"] = cls["product_group"].get("display_value")
            if cls.get("binding"):
                data["binding"] = cls["binding"].get("display_value")
        
        # Красивый вывод
        print(f"\n{'='*70}")
        print(f"📦 ОПИСАНИЕ ТОВАРА: {asin}")
        print(f"{'='*70}\n")
        print(f"📌 Title: {data['title']}")
        print(f"🏷️ Brand: {data['brand']}")
        if data['manufacturer'] and data['manufacturer'] != data['brand']:
            print(f"🏭 Manufacturer: {data['manufacturer']}")
        if data['edition']:
            print(f"📀 Edition: {data['edition']}")
        if data['product_group']:
            print(f"📂 Product Group: {data['product_group']}")
        
        if data['product_info']:
            print(f"\n📋 Product Info:")
            for k, v in data['product_info'].items():
                print(f"   • {k}: {v}")
        
        if data['technical_info']:
            print(f"\n🔧 Technical Info:")
            for k, v in data['technical_info'].items():
                print(f"   • {k}: {v}")
        
        if data['features']:
            print(f"\n✨ Features ({len(data['features'])} пунктов):")
            for i, f in enumerate(data['features'], 1):
                print(f"   {i}. {f[:100]}..." if len(f) > 100 else f"   {i}. {f}")
        
        return data
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return {}

print("✅ Функция get_product_description_data() готова!")

✅ Функция get_product_description_data() готова!


In [24]:
data = get_product_description_data("B0FKM2741R")


📦 ОПИСАНИЕ ТОВАРА: B0FKM2741R

📌 Title: Durskfl Mangiatoia intelligente per uccelli con Telecamera solare Bird Feeder Camera per uccellini da esterno stazione di alimentazione per 1080P HD automatica di uccelli/foto (Verde)
🏷️ Brand: Durskfl
📂 Product Group: Giardino e Terrazza

📋 Product Info:
   • color: Verde
   • unit_count: 1

✨ Features (7 пунктов):
   1. Nota: Questo dispositivo non supporta la visualizzazione in tempo reale sui telefoni cellulari
   2. Rilevamento del movimento PIR: Con un obiettivo HD 1080p e un grandangolo di 110°, scattare foto e v...
   3. Giorno e Notte: La mangiatoia per uccelli con visione notturna consente di continuare a guardare dop...
   4. Carica della Batteria e Ricarica Solare per Uccelli: La telecamera per uccelli con cavo USB ha una b...
   5. Mangiatoia Colibrì e Lavandino d'acqua: L'alimentatore per uccelli intelligente viene fornito con du...
   6. Facile da Installare: Goditi un'esperienza di birdwatching senza stress. La nostra telecamera 

# Найти Child IDs

In [4]:
# Проверим доступные ресурсы GetBrowseNodesResource
from amazon_paapi.sdk.models.get_browse_nodes_request import GetBrowseNodesRequest
from amazon_paapi.sdk.models.get_browse_nodes_resource import GetBrowseNodesResource

print("Доступные ресурсы GetBrowseNodesResource:")
for attr in dir(GetBrowseNodesResource):
    if not attr.startswith('_') and attr.isupper():
        print(f"  - {attr}")

Доступные ресурсы GetBrowseNodesResource:
  - ANCESTOR
  - CHILDREN


In [9]:
# Функция для получения дочерних категорий (Browse Nodes)

from amazon_paapi.sdk.models.get_browse_nodes_request import GetBrowseNodesRequest
from amazon_paapi.sdk.models.get_browse_nodes_resource import GetBrowseNodesResource

def get_category_children(browse_node_id: str) -> list:
    """
    Получает все дочерние категории (child nodes) для указанного browse_node_id.
    """
    api_client = DefaultApi(access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST, region=REGION)
    
    # Собираем все доступные ресурсы
    resources = []
    for attr in dir(GetBrowseNodesResource):
        if not attr.startswith('_') and attr.isupper():
            resources.append(getattr(GetBrowseNodesResource, attr))
    
    try:
        request = GetBrowseNodesRequest(
            partner_tag=PARTNER_TAG,
            partner_type=PartnerType.ASSOCIATES,
            marketplace="www.amazon.it",
            browse_node_ids=[browse_node_id],
            resources=resources,
            languages_of_preference=["it_IT"]
        )
        
        response = api_client.get_browse_nodes(request)
        result = response.to_dict()
        
        if not result or not result.get("browse_nodes_result"):
            print(f"❌ Категория не найдена: {browse_node_id}")
            return []
        
        nodes_result = result["browse_nodes_result"].get("browse_nodes", [])
        if not nodes_result:
            print(f"❌ Нет данных для категории: {browse_node_id}")
            return []
        
        node = nodes_result[0]
        node_name = node.get("display_name") or node.get("context_free_name", "Unknown")
        
        print(f"\n{'='*70}")
        print(f"📂 КАТЕГОРИЯ: {node_name} (ID: {browse_node_id})")
        print(f"{'='*70}\n")
        
        # Показываем предка
        if node.get("ancestor"):
            ancestor = node["ancestor"]
            ancestor_name = ancestor.get("display_name") or ancestor.get("context_free_name", "Unknown")
            print(f"⬆️ Родитель: {ancestor_name} (ID: {ancestor.get('id')})")
        
        # Показываем детей
        children = node.get("children", []) or []
        
        if not children:
            print(f"\n⚠️ Нет дочерних категорий (это leaf node)")
            return []
        
        print(f"\n📋 Дочерние категории ({len(children)}):\n")
        
        children_data = []
        for i, child in enumerate(children, 1):
            child_id = child.get("id")
            child_name = child.get("display_name") or child.get("context_free_name", "Unknown")
            
            children_data.append({"id": child_id, "name": child_name})
            print(f"   {i:2}. {child_name} (ID: {child_id})")
        
        print(f"\n{'='*70}")
        print("📋 Список ID для копирования:")
        print(f"   {[c['id'] for c in children_data]}\n")
        
        return children_data
        
    except Exception as e:
        print(f"❌ Error: {type(e).__name__}: {e}")
        return []

print("✅ Функция готова!")

✅ Функция готова!


In [26]:
children = get_category_children("2892907031") 


📂 КАТЕГОРИЯ: Calze e collant (ID: 2892907031)

⬆️ Родитель: Abbigliamento (ID: 2892859031)

📋 Дочерние категории (6):

    1. Autoreggenti (ID: 2893114031)
    2. Calze (ID: 2893117031)
    3. Calze a pantofola (ID: 2893115031)
    4. Calze per reggicalze (ID: 2893116031)
    5. Collant e calzamaglie (ID: 2893120031)
    6. Scaldamuscoli (ID: 2893121031)

📋 Список ID для копирования:
   ['2893114031', '2893117031', '2893115031', '2893116031', '2893120031', '2893121031']



# Поиск по Node

In [67]:
# Cell: Получить 20 товаров по Browse Node ID

def get_20_products_by_node(browse_node_id: str, min_rating: int = 4) -> list:
    """
    Получает 20 товаров по browse_node_id (2 страницы по 10).
    
    Args:
        browse_node_id: ID категории Amazon
        min_rating: Минимальный рейтинг (1-5)
    
    Returns:
        Список товаров (до 20 штук)
    """
    all_items = []
    
    for page in [1, 2]:
        print(f"📄 Загружаем страницу {page}...")
        
        response = search_api(
            browse_node_id=browse_node_id,
            item_page=page,
            min_reviews_rating=min_rating
        )
        
        if response and response.get("search_result") and response["search_result"].get("items"):
            items = response["search_result"]["items"]
            all_items.extend(items)
            print(f"   ✅ Получено {len(items)} товаров")
        else:
            print(f"   ⚠️ Нет товаров на странице {page}")
            break
        
        time.sleep(1)  # Rate limit
    
    # Выводим результат
    print(f"\n{'='*70}")
    print(f"📦 НАЙДЕНО {len(all_items)} ТОВАРОВ (node: {browse_node_id})")
    print(f"{'='*70}\n")
    
    for i, item in enumerate(all_items, 1):
        asin = item.get("asin", "N/A")
        title = item.get("item_info", {}).get("title", {}).get("display_value", "N/A")[:50]
        price, _ = extract_price_from_item(item)
        cr = item.get("customer_reviews") or {}
        sr = cr.get("star_rating") or {}
        rating = sr.get("value", "N/A")
        
        print(f"{i:2}. {asin} | {price:>10} | ⭐{rating} | {title}...")
    
    return all_items


# Пример использования:
# products = get_20_products_by_node("425916031")  # Electronics

In [69]:
products = get_20_products_by_node("3119615031", min_rating=4)

📄 Загружаем страницу 1...
⚠️ OffersV2 not available, using fallback: OFFERS_SUMMARIES_LOWESTPRICE
⚠️ OffersV2 not available, using fallback: OFFERS_LISTINGS_PRICE
⚠️ OffersV2 not available, using fallback: OFFERS_LISTINGS_CONDITION
⚠️ OffersV2 not available, using fallback: OFFERS_LISTINGS_ISBUYBOXWINNER
   ✅ Получено 10 товаров
📄 Загружаем страницу 2...
⚠️ OffersV2 not available, using fallback: OFFERS_SUMMARIES_LOWESTPRICE
⚠️ OffersV2 not available, using fallback: OFFERS_LISTINGS_PRICE
⚠️ OffersV2 not available, using fallback: OFFERS_LISTINGS_CONDITION
⚠️ OffersV2 not available, using fallback: OFFERS_LISTINGS_ISBUYBOXWINNER
   ✅ Получено 10 товаров

📦 НАЙДЕНО 20 ТОВАРОВ (node: 3119615031)

 1. B0BZMLKXL4 |    36,99 € | ⭐N/A | Tapo C510W Telecamera WiFi da esterno 2K 3MP, visi...
 2. B06ZZP99NJ |    70,50 € | ⭐N/A | U-Power | Scarpe Antinfortunistiche Basse Point S ...
 3. B00I46CIPM |    13,00 € | ⭐N/A | DUNLOP Protective Footwear, K580011, Dee, Verde, T...
 4. B0FP13P6L3 |    19,